## Example of looking at high rate event data from a strainmeter

In [ ]:
import numpy as np
import pandas as pd
import math
from earthscopestraintools.mseed_tools import ts_from_mseed
from earthscopestraintools.gtsm_metadata import GtsmMetadata
from earthscopestraintools.timeseries import Timeseries
from earthscopestraintools.event import Earthquake
from earthscopestraintools.event_processing import calc_hypocentral_dist, magnitude_plot, plot_coseismic_offset
from datetime import datetime, timedelta

import logging
logger = logging.getLogger()
logging.basicConfig(
        format="%(message)s", level=logging.INFO
    )

In [ ]:
network = 'IV'
station = 'TSM3'
meta = GtsmMetadata(network,station)

### Load event data based on USGS event_id

In [ ]:
eq = Earthquake(event_id = "us7000jiky")
#eq = Earthquake(event_id = "us7000jiim")
hypocentral_distance = calc_hypocentral_dist(eq.lat,
                                                 eq.long,
                                                 eq.depth,
                                                 meta.latitude,
                                                 meta.longitude)
print(f"USGS Magnitude {eq.mag} at {hypocentral_distance} km at {eq.time}") 
meta.get_event_terms()

### Load raw strain data

In [ ]:
period = 20
start = (eq.time - timedelta(seconds=15)).strftime("%Y-%m-%dT%H:%M:%S")
end = (eq.time + timedelta(seconds=60)).strftime("%Y-%m-%dT%H:%M:%S")
strain_raw = ts_from_mseed(network=network, station=station, location='T0', channel='BS*', start=start, end=end)
strain_raw.stats()
strain_raw.plot(type='line')

### Convert counts to microstrain

In [ ]:
gauge_microstrain = strain_raw.linearize(reference_strains=meta.reference_strains, gap=meta.gap)
gauge_microstrain.stats()


### Interpolate and high pass filter (1000s cutoff)

In [ ]:
gauge_microstrain_interpolated = gauge_microstrain.interpolate(method='linear', limit_seconds=3600)
gauge_microstrain_interpolated.stats()
#ts3.plot(type='line')

In [ ]:
gauge_microstrain_filtered = gauge_microstrain_interpolated.butterworth_filter(filter_type='high', filter_order=2, filter_cutoff_s=1000)
gauge_microstrain_filtered.stats()
gauge_microstrain_filtered.plot(type='line')

### Calculate dynamic strain and estimated magnitude using Barbour et al 2021

In [ ]:
dynamic_strain = gauge_microstrain_filtered.dynamic_strain()
dynamic_strain.stats()
dynamic_strain.plot(type='line')

In [ ]:
estimated_magnitude = dynamic_strain.calculate_magnitude(hypocentral_distance, meta.site_term, meta.longitude_term)
estimated_magnitude.plot()

In [ ]:
title = f"{network}.{station} at {hypocentral_distance} km from {eq.name}"
magnitude_plot(dynamic_strain_df=dynamic_strain.data, 
               magnitude_df=estimated_magnitude.data, 
               eq_time=eq.time, 
               eq_mag=eq.mag, 
               title=title)

### Plot any co-seismic offsets in regional strains

In [ ]:
regional_microstrain = gauge_microstrain.apply_calibration_matrix(meta.strain_matrices['lab_strain_matrix'])
regional_microstrain.plot()

In [ ]:
plot_coseismic_offset(
    df = regional_microstrain.data,
    plot_type='line',
    units = 'microstrain',
    eq_time= eq.time,
    coseismic_offset = True,
    color="black",)
    